In [117]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
keywordSet = set()
citiesSet = set()
countriesSet = set()
locationsSet = set()

In [118]:
def CorrectString(setWithPossibleKeywords, text, considerHashtag):
    word = 'Unknown'
    for each in setWithPossibleKeywords:
        if (each.lower() in text.lower()):
            word = each;
            break
    if (considerHashtag and word == 'Unknown'):
        for each in text.split():
            if (each.startswith('#')):
                word = each.lstrip('#')
    return word

In [119]:
def fixMissingFields(setWithPossibleKeywords, dataframe, nameOfColumnWithParamMissing = 'keyword', nameOfPlaceWithText = 'text', useHashtag = True): 
    for index, row in dataframe.iterrows():
        if (isinstance(row[nameOfColumnWithParamMissing], str)):
            continue
        dataframe.loc[index, nameOfColumnWithParamMissing] = CorrectString(setWithPossibleKeywords, row[nameOfPlaceWithText], useHashtag)


In [120]:
def addToSet(nameOfSet, dataFrameRows):
    for each in dataFrameRows:
        if (each == '' or each.isspace()):
            continue
        nameOfSet.add(each.lstrip().rstrip())

In [121]:
#Define the sets and texts
tweets = pd.read_csv('train.csv', encoding = 'latin-1')
dfWithKeywords = tweets.dropna(subset=['keyword'])
dfWithLocations = tweets.dropna(subset=['location'])
locations = pd.read_csv('worldcities.csv', encoding = 'latin-1')
addToSet(keywordSet, dfWithKeywords['keyword'])
addToSet(locationsSet, dfWithLocations['location'])
addToSet(citiesSet, locations['city'])
addToSet(countriesSet, locations['country'])
arrayWithSets = [locationsSet, citiesSet, countriesSet]

In [122]:
#Fix missing fields
fixMissingKeywords(keywordSet, tweets)
for eachSet in arrayWithSets:
    fixMissingKeywords(eachSet, tweets, 'location', 'text', False)

In [125]:
tweets['keyword']

0       earthquake
1             fire
2       evacuation
3       evacuation
4         wildfire
           ...    
7608      collapse
7609          fire
7610       volcano
7611        police
7612      wildfire
Name: keyword, Length: 7613, dtype: object